<div style="display: flex; align-items: center; justify-content: center;">
  <div style="display: flex; align-items: center;">
    <img src="templates/Other/logo.png" style="margin-right: 30px; border-radius: 10%;">
    <div>
      <h2>Karate Kido 2 Automation</h2>
      <h7 style="font-size: 18px">Computer Vision Project</h7>
      <p style="font-size: 13px";>By [Ghassan Jarbouh, Kosai Sheikh-Ali, Ali Saifo, Amir Kanhoosh]</p> 
    </div>
  </div>
</div>

In [33]:
# Imports
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QDesktopWidget, QPushButton
from PyQt5.QtGui import QPainter, QPen, QColor, QPixmap, QFont,QFontDatabase,QIcon
from concurrent.futures import ThreadPoolExecutor
from PyQt5.QtCore import Qt, QRect, QPoint
from pynput import keyboard
from PyQt5 import QtCore
from mss import mss
import numpy as np
import cv2 as cv
import threading
import pyautogui
import time
import sys
from sklearn.cluster import DBSCAN


In [34]:
# App Window GUI
start_automating = False
class AppWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.icon_size = 100  
        self.window_height = 320
        self.window_width = 500
        self.dragging = False  
        self.drag_start_pos = QPoint()
        self.setWindowTitle("Karate Kido 2 Automator")
        self.setWindowIcon(QIcon("templates/Other/karate_kido_icon.png"))
        self.setWindowFlags(Qt.FramelessWindowHint | QtCore.Qt.WindowStaysOnTopHint)
        desktop = QDesktopWidget()

        
        screen_width = desktop.screenGeometry().width()
        screen_height = desktop.screenGeometry().height()

        font_id = QFontDatabase.addApplicationFont(
            "templates/Other/FalconSportTwo.ttf")  
        font_families = QFontDatabase.applicationFontFamilies(font_id)
        if font_families:
            custom_font_family = font_families[0]
            self.title_font = QFont(custom_font_family, 18)
            self.status_font = QFont(custom_font_family, 16)
            self.close_font = QFont(custom_font_family, 12)

        self.setAttribute(Qt.WA_TranslucentBackground)

        
        self.setGeometry(screen_width // 2 - self.window_width // 2,
                         screen_height // 2 - self.window_height // 2,
                         self.window_width,
                         self.window_height)

        
        self.icon_label = QLabel(self)
        self.icon_label.setGeometry(
            self.width() // 2 - self.icon_size // 2, 0, self.icon_size, self.icon_size)

        pixmap = QPixmap("templates/Other/karate_kido_icon.png")
        scaled_pixmap = pixmap.scaled(
            self.icon_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
        self.icon_label.setPixmap(scaled_pixmap)

        self.text_label = QLabel(self)
        
        self.text_label.setGeometry(0, 110, self.width(), 50)

        
        self.text_label.setFont(self.title_font)
        self.text_label.setText("Karate Kido 2 Automator")
        self.text_label.setStyleSheet("color: white;")  

        
        self.text_label.setAlignment(Qt.AlignHCenter | Qt.AlignVCenter)

        horizontal_margin = 30  
        self.status_container = QLabel(self)
        self.status_container.setGeometry(
            horizontal_margin, 170, self.width() - 2 * horizontal_margin, 50
        )

        
        self.status_container.setStyleSheet("""
            background-color: rgba(255, 0, 0, 0.3);
            border-radius: 15px;
            color: white;
        """)
        self.status_container.setAlignment(Qt.AlignCenter)
        self.status_container.setFont(self.status_font)
        self.status_container.setText("Status: Not Ready")

        
        self.start_button = QPushButton("Start", self)
        self.start_button.setGeometry(horizontal_margin, 230, self.width() - 2 * horizontal_margin, 50)
        self.start_button.setFont(self.status_font)
        self.start_button.setStyleSheet("""
            background-color: rgba(3, 126, 229, 1);
            border-radius: 15px;
            color: white;
        """)
        self.start_button.clicked.connect(self.on_start_button_pressed)

        self.exit_text = QLabel(self)
        self.exit_text.setGeometry(0, 285, self.width(), 30)

        
        self.exit_text.setFont(self.close_font)
        self.exit_text.setText("Press 'esc' to Exit")
        self.exit_text.setStyleSheet("color: #adacac;")
        self.exit_text.setAlignment(Qt.AlignHCenter | Qt.AlignVCenter)
        

    def change_status(self, text: str, color: str):
        self.status_container.setText(f"Status: {text}")
        if "Ready" not in text and "Running" not in text:
            self.status_container.setStyleSheet(f"""
                background-color: rgba(0.3,0.3,0.3,1);
                border-radius: 15px;
                color: white;
            """)
            time.sleep(0.1)
        self.status_container.setStyleSheet(f"""
            background-color: {color};
            border-radius: 15px;
            color: white;
        """)

    def on_start_button_pressed(self):
        global start_automating
        self.start_button.setText("Press 'q' to Stop")
        self.start_button.setEnabled(False)
        self.start_button.setStyleSheet("""
            background-color: rgba(128, 128, 128, 0.7);
            border-radius: 15px;
            color: white;
        """)
        start_automating = True

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.dragging = True
            self.drag_start_pos = event.globalPos() - self.frameGeometry().topLeft()
            event.accept()

    def mouseMoveEvent(self, event):
        if self.dragging and event.buttons() == Qt.LeftButton:
            self.move(event.globalPos() - self.drag_start_pos)
            event.accept()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.dragging = False
            event.accept()

    def paintEvent(self, event):        
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        pen = QPen(QColor("black"), 0)
        painter.setPen(pen)
        painter.setBrush(QColor(50, 50, 50, 255))
        rect = QRect(0, self.icon_size // 2, self.width(),
                     self.height() - self.icon_size // 2)
        painter.drawRoundedRect(rect, 25, 25)
        
        painter.setBrush(QColor(0, 0, 0, 255))
        rect = QRect(5, 5+ self.icon_size // 2, self.width() -10,
                     self.height() - 10 - self.icon_size // 2)
        painter.drawRoundedRect(rect, 20, 20)
        
def get_properties_by_ID(ID):
    if ID == 'NBlanternRed':
        return "God Mode", "rgba(183, 49, 74, 0.8)"
    if ID == 'NBlanternGreen':
        return "Energy", "rgba(47, 120, 65, 0.8)"
    if ID == 'NBlanternBlue':
        return "Freeze", "rgba(64, 116, 153, 0.85)"
    if ID == 'NBlanternYellow':
        return "2x Score", "rgba(218, 203, 48, 0.9)"

In [35]:
# Controls
save_video_upon_exit = True
app_running = None
status = ""
fps = 30
window = None
game_detected = False

def on_press(key):
    global app_running, window, start_automating,game_detected
    try:
        if (key.char == 'q') and start_automating:
            window.start_button.setText("Start")
            window.start_button.setEnabled(True)
            window.start_button.setStyleSheet("""
                background-color: rgba(3, 126, 229, 1);
                border-radius: 15px;
                color: white;
            """)
            start_automating = False
            game_detected = False
    except AttributeError:
        if (key == keyboard.Key.esc) and app_running:
            app_running = False
            window.close()
        pass


kb = keyboard.Controller()
listener = keyboard.Listener(on_press=on_press)
listener.start()

In [36]:
# Main Logic Definitions

# - # reference constants # - #
MIN_DISTANCE_BETWEEN_TWO_BRANCHES_REFERENCE = 210
MIN_BRANCH_CENTER_DISTANCE_REFERENCE = 100
BRANCH_ABOVE_THRESHOLD_REFERENCE = 167
REFERENCE_DIMENSIONS = [913, 695]
CONST_MULTIPLIER_AHEAD = 0.63
TREE_WIDTH_REFERENCE = 90
TRUNK_CUT_REFERENCE = 105
PLAYER_Y_REFERENCE = 673
CONST_FREEZE_AHEAD = .6

# - # Delay constants # - #
DELAY_EVADE_IMPOSSIBLE_LANTERN = 2.5
DELAY_RESET_MULTIPLIERS_LIMIT = 0.8
DELAY_RESET_LANTERNS_LIMIT = 2
DELAY_RESET_FROZEN_LIMIT = 1
DELAY_GOD_MODE_LANTERN = 5
DELAY_LEVEL_UP_SCREEN = 5

# - # Automation Variables # - #
height = width = centerline = ratio_w = ratio_h = player_y = right_click = left_click = None
evade_impossible_lantern = False
automation_paused = False
lantern_detected = False
player_direction = "Left"
resized_templates = []

margins = {
    'branch': 5,
    'NB': 5
}

limits = {
    'NBlanternYellow': 1,
    'NBlanternGreen': 1,
    'NBlanternBlue': 1,
    'NBlanternRed': 1,
    'NBfrozen': 1,
    'NBx2': 1,
    'NBx3': 1,
    'NBx4': 1,
}

orb_templates = [
    # - # Lanterns # - #
    ('NBlanternRed',    cv.imread('templates/red_lantern_final.png', 0), 0.77, cv.ORB_create(nfeatures=750, nlevels=24, WTA_K=2,edgeThreshold = 0,fastThreshold = 11)),
    ('NBlanternGreen',  cv.imread('templates/final_green_lantern.png', 0), 0.8, cv.ORB_create(nfeatures=750, nlevels=24, WTA_K=2,edgeThreshold = 0,fastThreshold = 10)),
    ('NBlanternBlue',   cv.imread('templates/final_freeze.png', 0), 0.8, cv.ORB_create(nfeatures=750, nlevels=24, WTA_K=2,edgeThreshold = 0,fastThreshold = 14)),
    ('NBlanternYellow', cv.imread('templates/2x_lantern_final.png', 0), 0.8, cv.ORB_create(nfeatures=750, nlevels=24, WTA_K=2,edgeThreshold = 0,fastThreshold = 10)),
]

template_kp_des_orb = []
for info in orb_templates:
    template = info[1]
    kp, des = info[3].detectAndCompute(template, None)
    template_kp_des_orb.append((info[0],kp, des))

templates_data = [
    # - # Branches # - #
    ('right_branch', cv.imread('templates/right_branch9.png', 0), 0.86),
    ('right_branch', cv.imread('templates/right_branch6.png', 0), 0.89),
    ('right_branch', cv.imread('templates/right_branch10.png', 0), 0.89),
    ('right_branch', cv.imread('templates/right_branch7.png', 0), 0.89),
    ('right_branch', cv.imread('templates/right_branch3.png', 0), 0.89),
    ('right_branch', cv.imread('templates/right_branch2.png', 0), 0.89),
    ('left_branch',  cv.imread('templates/left_branch.png', 0), 0.87),

    # - # Frozen parts # - #
    ('NBfrozen', cv.imread('templates/frozen_with_number1.png', 0), 0.8),
    ('NBfrozen', cv.imread('templates/frozen2.png', 0), 0.94),
    ('NBfrozen', cv.imread('templates/frozen3.png', 0), 0.94),

    # # - # Lanterns # - #
    # ('NBlanternRed',    cv.imread('templates/red_lantern2.png', 0), 0.77),
    # ('NBlanternRed',    cv.imread('templates/red_lantern.png', 0), 0.77),
    # ('NBlanternGreen',  cv.imread('templates/lantern.png', 0), 0.8),
    # ('NBlanternBlue',   cv.imread('templates/freeze.png', 0), 0.8),
    # ('NBlanternYellow', cv.imread('templates/2x_2.png', 0), 0.8),
    # ('NBlanternYellow', cv.imread('templates/2x.png', 0), 0.8),

    # - # Level-up # - #
    ('NB_LEVEL_UP', cv.imread('templates/levelup.png', 0), 0.8),

    # - # Cutting Multipliers # - #
    ('NBx2', cv.imread('templates/x2.png', 0), 0.7),
    ('NBx3', cv.imread('templates/x3.png', 0), 0.7),
    ('NBx4', cv.imread('templates/x4.png', 0), 0.7),
]

In [37]:
# Game View Detection

def detect_game_view(frame, sift, template, kp_template, des_template):
    original_height, original_width = frame.shape[:2]
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    resized_width, resized_height = original_width // 2, original_height // 2
    gray_frame_resized = cv.resize(gray_frame, (resized_width, resized_height))

    kp_frame, des_frame = sift.detectAndCompute(gray_frame_resized, None)
    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
    matches = bf.match(des_template, des_frame)
    matches = sorted(matches, key=lambda x: x.distance)

    if len(matches) > 100:
        src_pts = np.float32([kp_template[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        matrix, _ = cv.findHomography(src_pts, dst_pts, cv.RHO, 0.35, confidence=0.995, maxIters=8000)

        if matrix is not None:
            h, w = template.shape
            points = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
            transformed_points = cv.perspectiveTransform(points, matrix)

            if cv.isContourConvex(transformed_points) and len(transformed_points) == 4:
                scaling_factor_x = original_width / resized_width
                scaling_factor_y = original_height / resized_height

                transformed_points[:, 0, 0] *= scaling_factor_x
                transformed_points[:, 0, 1] *= scaling_factor_y

                x1, y1 = transformed_points[0][0]  # Top-left
                # x2, y2 = transformed_points[1][0]  # Top-right
                x3, y3 = transformed_points[3][0]  # Bottom-left
                x4, y4 = transformed_points[2][0]  # Bottom-right
                
                # varying width
                transformed_points[0][0] = [x3,y1]
                transformed_points[1][0] = [x4,y1]
                
                # # + perfect square
                # transformed_points[3][0] = [x3, y3]  # Bottom-left (to be adjusted)
                # transformed_points[2][0] = [x4, y3]  # Bottom-right (to be adjusted)

                detected_width = np.linalg.norm(transformed_points[1][0] - transformed_points[0][0])
                detected_height = np.linalg.norm(transformed_points[3][0] - transformed_points[0][0])
                template_aspect_ratio = w / h
                if abs((detected_width / detected_height) - template_aspect_ratio) < 0.003 and detected_width > 0 and detected_height > 0:
                    # frame = cv.polylines(frame, [np.int32(transformed_points)], True, (0, 0, 255), 1)
                    size = (round(detected_width), round(detected_height))
                    return (int(transformed_points[0][0][0]), int(transformed_points[0][0][1])), size
    return None, None

In [38]:
# Main Logic

def click(click):
    pyautogui.click(x=click[0], y=click[1])


def continue_automating():
    global automation_paused
    automation_paused = False


def evading_succeeded():
    global evade_impossible_lantern
    evade_impossible_lantern = False


def reset_multipliers_limit():
    limits['NBx4'] = 1
    limits['NBx3'] = 1
    limits['NBx2'] = 1


def reset_lanterns_limit():
    limits['NBlanternYellow'] = 1
    limits['NBlanternGreen'] = 1
    limits['NBlanternBlue'] = 1
    limits['NBlanternRed'] = 1


def reset_frozen_limit():
    limits['NBfrozen'] = 1


def limit_multipliers_detection(clicks):
    for i in range(2, clicks + 1):
        limits[f'NBx{i}'] = 0


def limit_lanterns_detection():
    limits['NBlanternYellow'] = 0
    limits['NBlanternGreen'] = 0
    limits['NBlanternBlue'] = 0
    limits['NBlanternRed'] = 0


def limit_frozen_detection():
    limits['NBfrozen'] = 0


def resize_templates():
    return [(name, cv.resize(template, (int(template.shape[1] * ratio_w), int(template.shape[0] * ratio_h))), threshold)
            for name, template, threshold in templates_data]

def resize_orb_templates():
    return [(name, cv.resize(template, (int(template.shape[1] * ratio_w), int(template.shape[0] * ratio_h))), threshold, orb)
            for name, template, threshold, orb in orb_templates]


def filter_matches(locations):
    global automation_paused, status
    counts = {name: 0 for name in limits.keys()}
    filtered = []

    for loc in locations:
        template_name = loc['template']

        if 'LEVEL_UP' in template_name:
            status = "Level-UP Screen Delay.."
            window.change_status(text=status, color="rgba(202, 9, 6, 1)")

            automation_paused = True
            threading.Timer(DELAY_LEVEL_UP_SCREEN, continue_automating).start()
            break

        if 'branch' in template_name:
            if abs(loc['location'][0] - (centerline / 2)) < MIN_BRANCH_CENTER_DISTANCE_REFERENCE * ratio_w:
                continue
            margin = margins.get('branch', 5)
            
        elif 'NBfrozen' == template_name:
            if abs(loc['location'][0] - (centerline / 2)) > (TREE_WIDTH_REFERENCE + 20) * 0.5 * ratio_w:
                continue
            margin = margins.get('NB', 5)
            
        elif 'NB' in template_name and 'frozen' not in template_name:
            margin = margins.get('NB', 5)
            
        else:
            margin = 5

        if template_name in limits:
            if counts[template_name] < limits[template_name]:
                if all(np.linalg.norm(np.array(loc['location']) - np.array(f['location'])) > margin for f in filtered):
                    filtered.append(loc)
                    counts[template_name] += 1
        else:
            if all(np.linalg.norm(np.array(loc['location']) - np.array(f['location'])) > margin for f in filtered):
                filtered.append(loc)

    return filtered


def match_single_template(args):
    image, template, name, threshold = args
    if "left" in name:
        process_area = image[:, :image.shape[1] // 2]
    elif "right" in name:
        process_area = image[:, image.shape[1] // 2:]
    else:
        process_area = image

    result = cv.matchTemplate(process_area, template, cv.TM_CCOEFF_NORMED)
    locations = np.where(result >= threshold)
    matches = [{'template': name,
                'location': (pt[0] if (("left" in name) or ("NB" in name)) else pt[0] + image.shape[1] // 2, pt[1]),
                'size': template.shape}
               for pt in zip(*locations[::-1])]
    return matches


def check_clusters(matches, keypoints1, keypoints2, eps=50, min_samples=5):
    # Extract points from matches
    points = np.array([keypoints2[m.trainIdx].pt for m in matches])
    
    # Perform DBSCAN clustering
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(points)
    labels = clustering.labels_
    
    # Identify the largest cluster (ignoring noise points labeled as -1)
    unique_labels, counts = np.unique(labels, return_counts=True)
    largest_cluster_label = None
    max_cluster_size = 0

    # Find the largest cluster label and size
    for label, count in zip(unique_labels, counts):
        if label != -1 and count > max_cluster_size:
            largest_cluster_label = label
            max_cluster_size = count

    # Retrieve the first match in the largest cluster
    representative_match = None
    if largest_cluster_label is not None:
        cluster_indices = np.where(labels == largest_cluster_label)[0]
        if len(cluster_indices) > 0:
            representative_match = matches[cluster_indices[0]]  # Take the first match

    return max_cluster_size, representative_match

# Draw bounding box around the detected object
def draw_bounding_box(image, point, padding=20):
    x_min, y_min = int(point[0] - padding), int(point[1] - padding)
    x_max, y_max = int(point[0] + padding), int(point[1] + padding)

    cv.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
    return image

frame_orb = cv.ORB_create(nfeatures=3500)

# Object detection logic
def match_single_orb_template(args):
    frame, kp_frame, des_frame, kp_template, des_template, name = args
      
    if des_frame is None:
        return None

    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck= True)
    
    if des_template is None:
        return None

    matches = bf.match(des_template, des_frame)
    matches = sorted(matches, key=lambda x: x.distance)
    
    Threshold = 100
    min_samples = 20
    
    if name == "NBlanternBlue":
        Threshold = 40
    elif name == "NBlanternRed":
        Threshold = 28
    elif name == "NBlanternYellow":
        Threshold = 39
    elif name == "NBlanternGreen":
        Threshold = 26
    
    max_cluster_size, representative_match = check_clusters(matches, kp_template, kp_frame, eps=20, min_samples=min_samples)
    if max_cluster_size > Threshold: 
        print(f"{name} -> {max_cluster_size} > {Threshold}")
        frame_idx = representative_match.trainIdx 
        frame_point = kp_frame[frame_idx].pt
        return {"template":name,
                "location":(int(frame_point[0]) if (("left" in name) or ("NB" in name)) else int(frame_point[0] + frame.shape[1] // 2), int(frame_point[1])),
                "size":(20,20)}

    return None



def perform_detection(frame):
    global automation_paused, evade_impossible_lantern, status

    filtered_matches = frozen = bottom_most_lantern = None
    clicks_needed = 1
    
    cropped_frame = cv.cvtColor(frame[int(height * 0.10):int(height * 0.90),
                                      int(width * 0.25):int(width * 0.75)], cv.COLOR_BGR2GRAY)
    kp_frame, des_frame = frame_orb.detectAndCompute(cropped_frame, None)
    # add delay here to make sure that the branch under the lantern isn't covered OR check beforehand if it's close to the branch under it
    with ThreadPoolExecutor() as executor:
        args = [(cropped_frame, template, name, threshold)
                for name, template, threshold in resized_templates]
        all_matches = executor.map(match_single_template, args)
    
    with ThreadPoolExecutor() as executor:
        # frame, kp_template,des_template, name, = args
        args = [(cropped_frame ,kp_frame, des_frame, kp_template, des_template, name)
                for  name,kp_template, des_template in template_kp_des_orb]
        all_orb_matches = executor.map(match_single_orb_template, args)
    # all_orb_matches = []
    # for  name,kp_template, des_template in template_kp_des_orb:
    #     result = match_single_orb_template(cropped_frame ,kp_frame, des_frame, kp_template, des_template, name)
    #     if result:
    #         all_orb_matches.append(result)
    
    all_orb_matches = [match for match in all_orb_matches if match is not None]
    matches = [match for sublist in all_matches for match in sublist]
    matches.extend(all_orb_matches)
    
    filtered_matches = filter_matches(matches)

    if not filtered_matches:
        return filtered_matches, frozen, clicks_needed, bottom_most_lantern

    filtered_matches.sort(
        key=lambda m: m['location'][1], reverse=True)

    withoutNB = []

    for match in filtered_matches:
        match['location'] = (
            match['location'][0] + int(width * 0.25),
            match['location'][1] + int(height * 0.10)
        )
        if "branch" in match['template'] or "lantern" in match['template']:
            withoutNB.append(match)

    base_condition = len(
        withoutNB) > 2 and "branch" in withoutNB[2]['template'] and "NBlantern" in withoutNB[1]['template'] and "branch" in withoutNB[0]['template']
    if base_condition and abs(withoutNB[0]['location'][1] -
                              withoutNB[2]['location'][1]) <= 210 * ratio_h:

        top_branch_direction = withoutNB[2]['location'][0] > centerline
        bottom_branch_direction = withoutNB[0]['location'][0] > centerline
        same_direction = top_branch_direction == bottom_branch_direction
        if same_direction:
            status = "Skipping Impossible Lantern.."
            window.change_status(text=status, color="rgba(150, 3, 35, 1)")
            evade_impossible_lantern = True
            limit_lanterns_detection()

            threading.Timer(DELAY_EVADE_IMPOSSIBLE_LANTERN,
                            evading_succeeded).start()
            threading.Timer(DELAY_RESET_LANTERNS_LIMIT,
                            reset_lanterns_limit).start()

    if not evade_impossible_lantern and withoutNB and "NBlantern" in withoutNB[0]['template']:
        bottom_most_lantern = withoutNB[0]

    for loc in filtered_matches:
        if CONST_FREEZE_AHEAD <= loc['location'][1] / height and loc['location'][1] / height <= 0.8 and loc['template'] == 'NBfrozen':
            status = "Frozen Trunk Crushed!"
            window.change_status(text=status, color="rgba(159, 200, 210,1)")
            frozen = loc
            limit_frozen_detection()
            threading.Timer(DELAY_RESET_FROZEN_LIMIT,
                            reset_frozen_limit).start()

        if loc['location'][1] / height >= CONST_MULTIPLIER_AHEAD:
            if loc['template'] == 'NBx4':
                status = "x4 Multiplier Chopped!"
                window.change_status(
                    text=status, color="rgba(72, 79, 107,1)")
                clicks_needed = 4
                limit_multipliers_detection(clicks_needed)
                threading.Timer(DELAY_RESET_MULTIPLIERS_LIMIT,
                                reset_multipliers_limit).start()

            elif loc['template'] == 'NBx3':
                status = "x3 Multiplier Chopped!"
                window.change_status(
                    text=status, color="rgba(182, 145, 116,1)")
                clicks_needed = 3
                limit_multipliers_detection(clicks_needed)
                threading.Timer(DELAY_RESET_MULTIPLIERS_LIMIT,
                                reset_multipliers_limit).start()
            elif loc['template'] == 'NBx2':
                status = "x2 Multiplier Chopped!"
                window.change_status(
                    text=status, color="rgba(129, 114, 130,1)")
                clicks_needed = 2
                limit_multipliers_detection(clicks_needed)
                threading.Timer(DELAY_RESET_MULTIPLIERS_LIMIT,
                                reset_multipliers_limit).start()

        pt = loc['location']
        h, w = loc['size']
        bottom_right = (pt[0] + w, pt[1] + h)
        cv.rectangle(frame, pt, bottom_right, (255, 0, 0), 2)

    return filtered_matches, frozen, clicks_needed, bottom_most_lantern


def detect_objects(frame):
    global player_direction, lantern_detected, automation_paused, status, fps

    results = (perform_detection(frame))
    filtered_matches, frozen, clicks_needed, bottom_most_lantern = results

    if filtered_matches:
        branch_above = False
        for loc in filtered_matches:
            branch_direction = "Right" if loc['location'][0] > centerline else "Left"

            if branch_direction != player_direction or "branch" not in loc['template']:
                continue

            if abs(loc['location'][1] - player_y) <= BRANCH_ABOVE_THRESHOLD_REFERENCE * ratio_h:
                branch_above = True
                break

        if branch_above:
            player_direction = "Right" if player_direction == "Left" else "Left"

        else:
            if bottom_most_lantern and abs(bottom_most_lantern['location'][1] - player_y) <= 207 * ratio_h:
                name, color = get_properties_by_ID(
                    bottom_most_lantern['template'])
                status = f"{name} Lantern"
                window.change_status(text=status, color=color)
                lantern_detected = True
                limit_lanterns_detection()
                threading.Timer(DELAY_RESET_LANTERNS_LIMIT,
                                reset_lanterns_limit).start()

                player_direction = "Right" if bottom_most_lantern[
                    'location'][0] > centerline else "Left"

                for _ in range(clicks_needed):
                    click(right_click) if player_direction == "Right" else click(
                        left_click)
                if (frozen != None):
                    click(right_click) if player_direction == "Right" else click(
                        left_click)

                player_direction = "Left" if bottom_most_lantern[
                    'location'][0] > centerline else "Right"

                click(right_click) if player_direction == "Right" else click(
                    left_click)

                if ("Red" in bottom_most_lantern['template']):
                    automation_paused = True
                    threading.Timer(DELAY_GOD_MODE_LANTERN,
                                    continue_automating).start()
                    status = "God-Mode Activated!"
                    window.change_status(
                        text=status, color="rgba(183, 49, 74, 0.85)")

            if not lantern_detected:
                for _ in range(clicks_needed):
                    click(right_click) if player_direction == "Right" else click(
                        left_click)
                if (frozen != None):
                    click(right_click) if player_direction == "Right" else click(
                        left_click)
            lantern_detected = False
    return frame

In [39]:
# Automation Loop
app_running = True
processed_frames = []

sift = cv.SIFT_create(nfeatures=2500)

template = cv.imread("template.png", 0)
template = cv.resize(template, (template.shape[1] // 2, template.shape[0] // 2))

kp_template, des_template = sift.detectAndCompute(template, None)
game_view_position = None
game_view_size = None
def processing_loop():
    global app_running, game_detected, game_view_position, game_view_size,right_click,left_click,centerline,ratio_w,ratio_h,player_y,resized_templates,height,width,orb_templates,template_kp_des_orb
    with mss() as sct:
        screen = sct.monitors[1]
        while app_running:
            if not automation_paused:
                screenshot = sct.grab(screen)
                frame = np.array(screenshot)
                frame = frame[:, :, :3].astype(np.uint8)
                if not game_detected or not start_automating:
                    game_view_position, game_view_size = detect_game_view(
                        frame, sift, template, kp_template, des_template)
                        
                    game_detected = game_view_position != None and game_view_size != None and start_automating
                    
                    if game_view_position != None and game_view_size != None:
                        window.change_status(text="Ready",color="rgba(8, 36, 3,1)")
                        window.start_button.setEnabled(True)
                    else:
                        window.change_status(text="Not Ready",color="rgba(255,0,0,0.3)")
                        window.start_button.setEnabled(False)
                    if not game_detected:
                        continue
                    
                    window.change_status(text="Running",color="rgba(0,255,0,0.3)")
                    x_min, y_min = game_view_position
                    game_width, game_height = game_view_size
                    # print(f"error margin = {(game_width / game_height) - 0.7612267250821468}")
                    # Calculate the bottom-right corner
                    x_max = x_min + game_width
                    y_max = y_min + game_height
                    center_x = (x_min + x_max) // 2
                    center_y = (y_min + y_max) // 2
                    
                    right_click = (center_x + 100, center_y)
                    left_click = (center_x - 100, center_y)
                    
                    height, width, _ = frame[y_min:y_max, x_min:x_max].shape
                    centerline = width // 2
                    
                    ratio_w = width / REFERENCE_DIMENSIONS[1]
                    ratio_h = height / REFERENCE_DIMENSIONS[0]

                    player_y = PLAYER_Y_REFERENCE * ratio_h * 0.96
                    
                    resized_templates = resize_templates()
                
                cv.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
                detect_objects(frame[y_min:y_max, x_min:x_max])
                
                if save_video_upon_exit:
                    processed_frames.append(frame)
                
                time.sleep(1/fps)

def save_video(frames):
    if frames:
        height, width, _ = frames[0].shape
        fourcc = cv.VideoWriter_fourcc(*"mp4v")
        output_file = "output.mp4"
        video_writer = cv.VideoWriter(output_file, fourcc, fps // 2, (width, height))
        print(f"saving video with {len(frames)} fps")

        for frame in frames:
            video_writer.write(frame)

        video_writer.release()
        print(f"Video saved: {output_file}")

In [40]:
# Launch Application

processing_thread = threading.Thread(target=processing_loop, daemon=True)
processing_thread.start()

if QApplication.instance() is None:
    app = QApplication(sys.argv)
else:
    app = QApplication.instance()

window = AppWindow()
window.show()
app.exec_()

if save_video_upon_exit:
    save_video(processed_frames)

NBlanternGreen -> 30 > 26
NBlanternBlue -> 44 > 40
NBlanternGreen -> 30 > 26
NBlanternGreen -> 38 > 26
NBlanternGreen -> 40 > 26
NBlanternGreen -> 33 > 26
NBlanternGreen -> 29 > 26
NBlanternGreen -> 29 > 26
NBlanternGreen -> 28 > 26
NBlanternGreen -> 27 > 26
NBlanternGreen -> 27 > 26
NBlanternBlue -> 50 > 40
NBlanternBlue -> 84 > 40
NBlanternBlue -> 73 > 40
NBlanternBlue -> 77 > 40
NBlanternBlue -> 64 > 40
NBlanternBlue -> 67 > 40
NBlanternBlue -> 56 > 40
NBlanternBlue -> 58 > 40
NBlanternBlue -> 62 > 40
NBlanternBlue -> 52 > 40
NBlanternBlue -> 45 > 40
NBlanternGreen -> 31 > 26
NBlanternGreen -> 34 > 26
NBlanternBlue -> 44 > 40
NBlanternGreen -> 28 > 26
NBlanternGreen -> 36 > 26
NBlanternGreen -> 42 > 26
saving video with 330 fps
Video saved: output.mp4
